#### Abidzar Muhammad Ghifary Kurniawan
# **Capstone Project**

### **Clustering neighborhoods of New York, Toronto and San Francisco**



before we start exploring, let's import the library that we will need.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt

from bs4 import BeautifulSoup #library to scrape the websites

# import k-means from clustering stage
from sklearn.cluster import KMeans
from scipy import spatial

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

from geopy import geocoders  # if there isn't coordinate in json file (San Francisco dataset)
gn = geocoders.GeoNames(username='abidzar')
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut

print('Libraries imported.')

Libraries imported.


## 1. Download and Explore Dataset

In order to segment the neighborhoods and explore them, we will essentially need a dataset that contains boroughs and the neighborhoods that exist in each borough as well as the the latitude and longitude coordinates of each neighborhood.
 
For each city, we will get dataset separately and join them at the end.

### 1.1 San Francisco Dataset

For this dataset, we will make it from only one sources : https://data.sfgov.org/api/views/6ia5-2f8k/rows.json?accessType=DOWNLOAD

#### 1.1.1 San Francisco Government

In [2]:
!wget -q -O 'sanfran_data.json' https://data.sfgov.org/api/views/6ia5-2f8k/rows.json?accessType=DOWNLOAD
print('Data downloaded!')

Data downloaded!


In [3]:
with open('sanfran_data.json') as json_data:
    SanFrancisco_data = json.load(json_data)
    print('Success')

Success


###### Tranform the data into a pandas dataframe
The next task is essentially transforming this data of nested Python dictionaries into a pandas dataframe. So let's start by creating an empty dataframe and find the coordinates using geocoders

In [4]:
neighboorhood = SanFrancisco_data['data']

In [5]:
#define a function to prevent GeocoderTimedOut error
def do_geocode(neighborhood_name):
    try:
        return gn.geocode("{}, CA, US".format(neighborhood_name),exactly_one=True)
    except GeocoderTimedOut:
        return do_geocode(neighborhood_name)

In [6]:
# define the dataframe columns
column_names = ['Neighborhood', 'Latitude', 'Longitude']

# instantiate the dataframe
san_fran = pd.DataFrame(columns=column_names)

#find the coordinates using geocoders
for data in neighboorhood:
    neighborhood_name = data[10]
    
    neighborhood_latlon = do_geocode('{}, CA, US'.format(neighborhood_name))
    
    if neighborhood_latlon == None :
        print('{} has no coordinate'.format(neighborhood_name))
    else :
        neighborhood_lat = neighborhood_latlon[1][0]
        neighborhood_lon = neighborhood_latlon[1][1]
        #append it to dataframe
        san_fran = san_fran.append({'Neighborhood': neighborhood_name,
                                                          'Latitude': neighborhood_lat,
                                                          'Longitude': neighborhood_lon},ignore_index=True)

Presidio National Park has no coordinate
Lincoln Park / Ft. Miley has no coordinate
Outer Richmond has no coordinate
Laurel Heights / Jordan Park has no coordinate
Downtown / Union Square has no coordinate
Mint Hill has no coordinate
Parkmerced has no coordinate
Lower Nob Hill has no coordinate
Upper Market has no coordinate
Dolores Heights has no coordinate
Peralta Heights has no coordinate
Ingleside Terraces has no coordinate
Mt. Davidson Manor has no coordinate
Candlestick Point SRA has no coordinate
Apparel City has no coordinate
Produce Market has no coordinate
Aquatic Park / Ft. Mason has no coordinate
Polk Gulch has no coordinate
Parnassus Heights has no coordinate


it seems that geocoders can't find several neighborhood's coordinate, but thats okay, 

NOTE : there is no 'Borough' in San Francisco, so we will fill borough with san francisco

In [7]:
#dont forget to input the city and borough
san_fran['City'] = 'San Francisco'
san_fran['Borough'] = 'San Francisco'

In [8]:
print('The dataframe has {} neighborhoods.'.format(
        san_fran.shape[0]
    )
)

#lets take a look to San Francisco Dataset
san_fran.head()

The dataframe has 98 neighborhoods.


,Neighborhood,Latitude,Longitude,City,Borough
0,Seacliff,36.97412,-121.91579,San Francisco,San Francisco
1,Lake Street,39.04700,-122.91418,San Francisco,San Francisco
2,Presidio Terrace,37.78826,-122.46080,San Francisco,San Francisco
3,Inner Richmond,37.90187,-122.34108,San Francisco,San Francisco
4,Sutro Heights,37.77826,-122.51108,San Francisco,San Francisco


### 1.2 Toronto Dataset

For this dataset, we will make it from two sources : wikipedia and cognitive lab

#### 1.2.1 Cognitive Lab
Cognitive lab has the list of coordinates of toronto based on postal code, so we will scrape it from there

In [10]:
#use pandas to convert csv into dataframe
url = 'http://cocl.us/Geospatial_data'
df_coordinates = pd.read_csv(url)

In [11]:
#set Postal code as an index
df_coordinates.set_index('Postal Code', inplace = True)

In [12]:
print('The dataframe has {} neighborhoods.'.format(df_coordinates.shape[0]))

#lets take a look to Toronto Dataset
df_coordinates.head()

The dataframe has 103 neighborhoods.


,Latitude,Longitude
Postal Code,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476


#### 1.2.2 Wikipedia
A wikipedia page has the list of neighborhoods of toronto based on postal code, so we will scrape it from there

In [13]:
src = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

In [14]:
#scrape it using BeautifulSoup
soup = BeautifulSoup(src.content, 'lxml')
html_tab = soup.find('table', class_='wikitable sortable')
toronto_list = pd.read_html(str(html_tab), header = 0)[0]
toronto_draft = pd.DataFrame(toronto_list)

In [15]:
#remove the rows that don't that borough = Not assigned
toronto_draft = toronto_draft[toronto_draft['Borough'] != 'Not assigned']
toronto_draft.reset_index(drop=True, inplace=True)

In [16]:
#unified the rows that have same Postcode
grouped = toronto_draft.groupby(['Postcode','Borough'],sort=False)['Neighbourhood'].apply(lambda x: ', '.join(x))
toronto_df = pd.DataFrame(grouped)
toronto_df.reset_index(inplace=True)
toronto_df['Neighbourhood'].replace('Not assigned',toronto_df['Borough'],inplace=True)

#set Postal code as an index
toronto_df.set_index('Postcode', inplace = True)

In [17]:
print('The dataframe has {} neighborhoods.'.format(toronto_df.shape[0]))

#lets take a look to Toronto Dataset
toronto_df.head()

The dataframe has 103 neighborhoods.


,Borough,Neighbourhood
Postcode,,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,"Harbourfront, Regent Park"
M6A,North York,"Lawrence Heights, Lawrence Manor"
M7A,Queen's Park,Queen's Park


#### 1.2.3 Combine two dataframe
concat two dataframe using several function from pandas library

In [18]:
#concat the dataframes
toronto = pd.concat([toronto_df, df_coordinates], axis=1, join='inner', join_axes=[toronto_df.index])

In [19]:
#reset the index, rename the column, and reset the index
toronto.reset_index(inplace = True, drop = True)
toronto.rename(columns={"Neighbourhood": "Neighborhood"}, inplace = True)
toronto['City'] = 'Toronto'
toronto.set_index('City', inplace = True)
toronto.reset_index(inplace = True)


In [20]:
print('The dataframe has {} neighborhoods.'.format(toronto.shape[0]))

#lets take a look to Toronto Dataset
toronto.head()

The dataframe has 103 neighborhoods.


,City,Borough,Neighborhood,Latitude,Longitude
0,Toronto,North York,Parkwoods,43.753259,-79.329656
1,Toronto,North York,Victoria Village,43.725882,-79.315572
2,Toronto,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,Toronto,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,Toronto,Queen's Park,Queen's Park,43.662301,-79.389494


### 1.3 New York Dataset

For this dataset, we will make it from two sources : https://geo.nyu.edu/catalog/nyu_2451_34572

In [21]:
!wget -q -O 'newyork_data.json' https://ibm.box.com/shared/static/fbpwbovar7lf8p5sgddm06cgipa2rxpe.json
print('Data downloaded!')

Data downloaded!


build the new dataframe from json file

In [22]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [23]:
ny_data = newyork_data['features']

In [24]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
new_york = pd.DataFrame(columns=column_names)

# let's loop through the data and fill the dataframe one row at a time
for data in ny_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    new_york = new_york.append({'Borough': borough,
                                              'Neighborhood': neighborhood_name,
                                              'Latitude': neighborhood_lat,
                                              'Longitude': neighborhood_lon}, ignore_index=True)



In [25]:
new_york['City'] = 'New York'
new_york.set_index('City', inplace = True)
new_york.reset_index(inplace = True)

In [26]:
print('The dataframe has {} neighborhoods.'.format(new_york.shape[0]))

#lets take a look to New York Dataset
new_york.head()

The dataframe has 306 neighborhoods.


,City,Borough,Neighborhood,Latitude,Longitude
0,New York,Bronx,Wakefield,40.894705,-73.847201
1,New York,Bronx,Co-op City,40.874294,-73.829939
2,New York,Bronx,Eastchester,40.887556,-73.827806
3,New York,Bronx,Fieldston,40.895437,-73.905643
4,New York,Bronx,Riverdale,40.890834,-73.912585


## 2. Segmenting and Clustering



 ### 2.1 Merging the Dataframes
 Use geopy library to get the latitude and longitude values of the three cities (just like when we find san francisco's neighborhood), after that, we will create a list of maps of three cities using Folium library

In [27]:
all = pd.concat([new_york, toronto, san_fran], ignore_index=True,sort=True)

In [28]:
print('The dataframe has {} neighborhoods.'.format(all.shape[0]))

#lets take a look to New York Dataset
all.head()

The dataframe has 507 neighborhoods.


,Borough,City,Latitude,Longitude,Neighborhood
0,Bronx,New York,40.894705,-73.847201,Wakefield
1,Bronx,New York,40.874294,-73.829939,Co-op City
2,Bronx,New York,40.887556,-73.827806,Eastchester
3,Bronx,New York,40.895437,-73.905643,Fieldston
4,Bronx,New York,40.890834,-73.912585,Riverdale


### 2.2 Find Nearby Venues
Define Foursquare Credentials and Version

In [29]:
CLIENT_ID = 'ANFWOW0XZ0KKQ54MEUOH33051Z5GPXVLKQJ4HW35AMZNMI2Y' # your Foursquare ID
CLIENT_SECRET = 'X1GNPFDFGLHZJ5JVBIVMLHGFXRA1OD1PP14GMAJK2FLS10Y3' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

define a function to get the nearby venues of Neighborhoods

In [30]:
LIMIT = 50

def getNearbyVenues(cities, names, latitudes, longitudes, radius=500):
    i=0
    venues_list=[]
    for city, name, lat, lng in zip(cities, names, latitudes, longitudes):
        i+=1
        print(i, city, name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            city,
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['City',
                  'Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [31]:
venues = getNearbyVenues(cities = all['City'], names = all['Neighborhood'],
                                   latitudes = all['Latitude'],
                                   longitudes = all['Longitude'])

1 New York Wakefield
2 New York Co-op City
3 New York Eastchester
4 New York Fieldston
5 New York Riverdale
6 New York Kingsbridge
7 New York Marble Hill
8 New York Woodlawn
9 New York Norwood
10 New York Williamsbridge
11 New York Baychester
12 New York Pelham Parkway
13 New York City Island
14 New York Bedford Park
15 New York University Heights
16 New York Morris Heights
17 New York Fordham
18 New York East Tremont
19 New York West Farms
20 New York High  Bridge
21 New York Melrose
22 New York Mott Haven
23 New York Port Morris
24 New York Longwood
25 New York Hunts Point
26 New York Morrisania
27 New York Soundview
28 New York Clason Point
29 New York Throgs Neck
30 New York Country Club
31 New York Parkchester
32 New York Westchester Square
33 New York Van Nest
34 New York Morris Park
35 New York Belmont
36 New York Spuyten Duyvil
37 New York North Riverdale
38 New York Pelham Bay
39 New York Schuylerville
40 New York Edgewater Park
41 New York Castle Hill
42 New York Olinville
43

In [32]:
# backup
# backup_v = venues

In [33]:
# recovery
# venues = backup_v

In [34]:
venues.head()

,City,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,New York,Wakefield,40.894705,-73.847201,Lollipops Gelato,40.894123,-73.845892,Dessert Shop
1,New York,Wakefield,40.894705,-73.847201,Rite Aid,40.896521,-73.844680,Pharmacy
2,New York,Wakefield,40.894705,-73.847201,Carvel Ice Cream,40.890487,-73.848568,Ice Cream Shop
3,New York,Wakefield,40.894705,-73.847201,Dunkin Donuts,40.890631,-73.849027,Donut Shop
4,New York,Wakefield,40.894705,-73.847201,SUBWAY,40.890656,-73.849192,Sandwich Place


In [35]:
# NOTE : Since we are using free foursquare API (which mean we only have limited use per day)
# it would be better if we export it to csv file, to prevent redudant foursquare API usage
# venues.to_csv('venues.csv', encoding='utf-8', index=True)

In [36]:
# checkpoint (you can start from here)
venues = pd.read_csv("venues.csv",index_col=0)

#### Let's check the size of the resulting dataframe

In [37]:
print('The dataframe has {} neighborhoods.'.format(venues.shape[0]))

#lets take a look to venues Dataset
venues.head()

The dataframe has 12118 neighborhoods.


,City,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,New York,Wakefield,40.894705,-73.847201,Lollipops Gelato,40.894123,-73.845892,Dessert Shop
1,New York,Wakefield,40.894705,-73.847201,Rite Aid,40.896521,-73.844680,Pharmacy
2,New York,Wakefield,40.894705,-73.847201,Carvel Ice Cream,40.890487,-73.848568,Ice Cream Shop
3,New York,Wakefield,40.894705,-73.847201,Dunkin Donuts,40.890631,-73.849027,Donut Shop
4,New York,Wakefield,40.894705,-73.847201,SUBWAY,40.890656,-73.849192,Sandwich Place


Let's check how many venues were returned for each neighborhood

In [38]:
venues.groupby('Neighborhood').count().head()

,City,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,,
"Adelaide, King, Richmond",50,50,50,50,50,50,50
Agincourt,4,4,4,4,4,4,4
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",2,2,2,2,2,2,2
Alamo Square,50,50,50,50,50,50,50
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",10,10,10,10,10,10,10


Let's find out how many unique categories can be curated from all the returned venues

In [39]:
print('There are {} uniques categories.'.format(len(venues['Venue Category'].unique())))

There are 467 uniques categories.


### 2.3 Analyze Each Neighborhood

In [40]:
# backup checkpoint
backup_2 = venues

In [41]:
# recovery checkpoint
venues = backup_2

In [42]:
# one hot encoding
venues_onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")

# add  neighborhood and city column back to dataframe
venues_onehot['Neighborhood'] = venues['Neighborhood']
venues_onehot['City'] = venues['City']

# move neighborhood and city column to the first column
# fixed_columns = [venues_onehot.columns[-1]] + list(venues_onehot.columns[:-1])
# venues_onehot = venues_onehot[fixed_columns]

In [43]:
City = venues_onehot['City']
venues_onehot.drop(labels=['City'], axis=1,inplace = True)
venues_onehot.insert(0, 'City', City)

Neighborhood = venues_onehot['Neighborhood']
venues_onehot.drop(labels=['Neighborhood'], axis=1,inplace = True)
venues_onehot.insert(0, 'Neighborhood', Neighborhood)

venues_onehot.head()

,Neighborhood,City,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,Airport Tram,American Restaurant,Animal Shelter,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Bath House,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cha Chaan Teng,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Christmas Market,Church,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Auditorium,College Basketball Court,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Curling Ice,Cycle Studio,Czech Restaurant,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Distillery,Dive Bar,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Elementary School,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Hakka Restaurant,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Hill,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Hunan Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Insurance Office,Intersection,Irish Pub,Island,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Kosher Restaurant,Lake,Laser Tag,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Lebanese Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Marijuana Dispensary,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Middle School,Mini Golf,Miscellaneous

And let's examine the new dataframe size.

In [44]:
venues_onehot.shape

(12118, 468)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [45]:
venues_grouped = venues_onehot.groupby('Neighborhood').mean().reset_index()
venues_grouped.head()

,Neighborhood,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,Airport Tram,American Restaurant,Animal Shelter,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Bath House,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cha Chaan Teng,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Christmas Market,Church,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Auditorium,College Basketball Court,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Curling Ice,Cycle Studio,Czech Restaurant,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Distillery,Dive Bar,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Elementary School,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Hakka Restaurant,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Hill,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Hunan Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Insurance Office,Intersection,Irish Pub,Island,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Kosher Restaurant,Lake,Laser Tag,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Lebanese Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Marijuana Dispensary,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Middle School,Mini Golf,Miscellaneous Shop

#### Let's confirm the new size

In [46]:
venues_grouped.shape

(489, 467)

#### build the new dataframe and fill it with each neighborhood along with the top 10 most common venues

First, let's write a function to sort the venues in descending order.

In [47]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [48]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = venues_grouped['Neighborhood']

for ind in np.arange(venues_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(venues_grouped.iloc[ind, :], num_top_venues)



In [49]:
neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Steakhouse,Café,American Restaurant,Coffee Shop,Breakfast Spot,Restaurant,Hotel,Bar,Asian Restaurant,Gastropub
1,Agincourt,Skating Rink,Breakfast Spot,Lounge,Clothing Store,Yoga Studio,Farmers Market,English Restaurant,Ethiopian Restaurant,Event Service,Event Space
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Park,Playground,Yoga Studio,Farmers Market,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit
3,Alamo Square,Bar,Sushi Restaurant,Wine Bar,Pizza Place,Seafood Restaurant,Mediterranean Restaurant,Cocktail Bar,Rock Club,BBQ Joint,Roller Rink
4,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Coffee Shop,Fried Chicken Joint,Pharmacy,Video Store,Fast Food Restaurant,Sandwich Place,Pizza Place,Beer Store,Yoga Studio


### 2.3 Cluster Neighborhoods

Run k-means to cluster the neighborhood into 5 clusters.

In [50]:
# set number of clusters
kclusters = 5

venues_clustering = venues_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=42).fit(venues_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 3, 4, 3, 0, 0, 0, 3, 0, 2], dtype=int32)

In [51]:
venues_merged = neighborhoods_venues_sorted

# add clustering labels
venues_merged['Cluster Labels'] = kmeans.labels_

# merge df_grouped with city data to add latitude/longitude for each neighborhood
venues_merged = venues_merged.join(all.set_index('Neighborhood'), on='Neighborhood')

venues_merged.head() # check the last columns!

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels,Borough,City,Latitude,Longitude
0,"Adelaide, King, Richmond",Steakhouse,Café,American Restaurant,Coffee Shop,Breakfast Spot,Restaurant,Hotel,Bar,Asian Restaurant,Gastropub,3,Downtown Toronto,Toronto,43.650571,-79.384568
1,Agincourt,Skating Rink,Breakfast Spot,Lounge,Clothing Store,Yoga Studio,Farmers Market,English Restaurant,Ethiopian Restaurant,Event Service,Event Space,3,Scarborough,Toronto,43.794200,-79.262029
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Park,Playground,Yoga Studio,Farmers Market,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit,4,Scarborough,Toronto,43.815252,-79.284577
3,Alamo Square,Bar,Sushi Restaurant,Wine Bar,Pizza Place,Seafood Restaurant,Mediterranean Restaurant,Cocktail Bar,Rock Club,BBQ Joint,Roller Rink,3,San Francisco,San Francisco,37.776290,-122.434670
4,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Coffee Shop,Fried Chicken Joint,Pharmacy,Video Store,Fast Food Restaurant,Sandwich Place,Pizza Place,Beer Store,Yoga Studio,0,Etobicoke,Toronto,43.739416,-79.588437


In [52]:
venues_merged['Cluster Labels'].value_counts()

3    356
0     96
4     24
2     21
1      3
Name: Cluster Labels, dtype: int64

In [53]:
# it would be better if we export it to csv file, to prevent redudant process
venues_merged.to_csv('venues_merged.csv', encoding='utf-8', index=True)

In [54]:
# checkpoint (you can start from here too)
# venues_merged = pd.read_csv("venues_merged.csv",index_col=0)

## 3. Visualize on the map
Finally, let's visualize the resulting clusters

In [55]:
addresses = ['San Francisco','Toronto, Ontario, Canada','New York City, NY']
latitude = []
longitude = []
geolocator = Nominatim()

for address in addresses:
    location = geolocator.geocode(address)
    latitude.append(location.latitude)
    longitude.append(location.longitude)
    print('The geograpical coordinate of {a} are {l}'.format(a=address, l=location[1]))

The geograpical coordinate of San Francisco are (37.7792808, -122.4192363)
The geograpical coordinate of Toronto, Ontario, Canada are (43.653963, -79.387207)
The geograpical coordinate of New York City, NY are (40.7308619, -73.9871558)


In [56]:
#this function is made for easier task
def map_code(x):
    return {
        'San Francisco': 0,
        'Toronto': 1,
        'New York': 2,
    }[x]

In [57]:
venues_merged['City'].value_counts()

New York         306
Toronto          100
San Francisco     94
Name: City, dtype: int64

In [58]:
map_clusters = []
for i in range(len(latitude)):
    map_clusters.append(folium.Map(location=[latitude[i], longitude[i]], zoom_start=10))

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, city, cluster in zip(venues_merged['Latitude'], venues_merged['Longitude'], venues_merged['Neighborhood'], venues_merged['City'], venues_merged['Cluster Labels']):
    label = folium.Popup(str(poi)+', '+ str(city) + ', Cluster ' + str(cluster), parse_html=True)
    
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters[map_code(city)])

after that, lets take a look on the maps we built

In [59]:
map_clusters[0] #San Francisco

In [60]:
map_clusters[1] #Toronto

In [61]:
map_clusters[2] #New York

Analysis for the project

In [62]:
clusters_total = venues_merged[['City','Cluster Labels']].groupby('City',sort=False).count().rename(columns={'Cluster Labels':'Total'}).transpose()
clusters_total.columns.name = None
clusters_total

df_desc = pd.concat([venues_merged[['City','Cluster Labels']].groupby('City').get_group(city).groupby('Cluster Labels').count().rename(
        columns={'City':city}) for city in venues_merged['City'].unique().tolist()], axis=1, sort=False)
df_desc = df_desc.append(clusters_total).astype(float)
df_desc

,Toronto,San Francisco,New York
0,17.0,3.0,76.0
1,1.0,1.0,1.0
2,1.0,1.0,19.0
3,71.0,81.0,204.0
4,10.0,8.0,6.0
Total,100.0,94.0,306.0


Now, we will find the similarities between cities by implementing Cosine similarity test

In [63]:
#for easier task, convert the dataframe into percentage
df_perc = df_desc/df_desc.iloc[5]
df_perc

,Toronto,San Francisco,New York
0,0.17,0.031915,0.248366
1,0.01,0.010638,0.003268
2,0.01,0.010638,0.062092
3,0.71,0.861702,0.666667
4,0.10,0.085106,0.019608
Total,1.00,1.000000,1.000000


In [64]:
list_sim = []
list_cities = []

for city1 in df_perc.columns:
    for city2 in df_perc.columns[df_perc.columns.tolist().index(city1)+1:]:
        result = 1 - spatial.distance.cosine(df_perc.iloc[:5][city1], df_perc.iloc[:5][city2])
        list_sim.append(result)
        list_cities.append(city1+' & '+city2)

df_similarity = pd.DataFrame({'Similarity':list_sim}, index=list_cities)
df_similarity

,Similarity
Toronto & San Francisco,0.980031
Toronto & New York,0.984114
San Francisco & New York,0.944505
